<a href="https://www.kaggle.com/code/zittagustavo/xgboost-spaceship-titanic?scriptVersionId=123049605" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
import numpy as np

# Tratamento dos dados

In [2]:
#importando o dataset
dados_treino= pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
dados_treino.set_index('PassengerId', inplace=True)
dados_treino.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
PassengerId,,,,,,,,,,,,,
0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [3]:
#pegando as informaçoes dos dados
dados_treino.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8693 entries, 0001_01 to 9280_02
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    8492 non-null   object 
 1   CryoSleep     8476 non-null   object 
 2   Cabin         8494 non-null   object 
 3   Destination   8511 non-null   object 
 4   Age           8514 non-null   float64
 5   VIP           8490 non-null   object 
 6   RoomService   8512 non-null   float64
 7   FoodCourt     8510 non-null   float64
 8   ShoppingMall  8485 non-null   float64
 9   Spa           8510 non-null   float64
 10  VRDeck        8505 non-null   float64
 11  Name          8493 non-null   object 
 12  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(6)
memory usage: 891.4+ KB


In [4]:
#ver os valores unicos
dados_treino.describe(include=('object','bool'))

,HomePlanet,CryoSleep,Cabin,Destination,VIP,Name,Transported
count,8492,8476,8494,8511,8490,8493,8693
unique,3,2,6560,3,2,8473,2
top,Earth,False,G/734/S,TRAPPIST-1e,False,Gollux Reedall,True
freq,4602,5439,8,5915,8291,2,4378


In [5]:
#vendo os nulos
dados_treino.isna().sum()

HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [6]:
#vendo quantos e quais sao os valores unicos de cada dimensao
cols = dados_treino.columns
for col in cols:
    if dados_treino[col].dtype != 'float64':
        print(dados_treino[col].value_counts())

Earth     4602
Europa    2131
Mars      1759
Name: HomePlanet, dtype: int64
False    5439
True     3037
Name: CryoSleep, dtype: int64
G/734/S     8
G/109/P     7
B/201/P     7
G/1368/P    7
G/981/S     7
           ..
G/556/P     1
E/231/S     1
G/545/S     1
G/543/S     1
F/947/P     1
Name: Cabin, Length: 6560, dtype: int64
TRAPPIST-1e      5915
55 Cancri e      1800
PSO J318.5-22     796
Name: Destination, dtype: int64
False    8291
True      199
Name: VIP, dtype: int64
Gollux Reedall        2
Elaney Webstephrey    2
Grake Porki           2
Sus Coolez            2
Apix Wala             2
                     ..
Jamela Griffy         1
Hardy Griffy          1
Salley Mckinn         1
Mall Frasp            1
Propsh Hontichre      1
Name: Name, Length: 8473, dtype: int64
True     4378
False    4315
Name: Transported, dtype: int64


## tratando os valores nulos

In [7]:
#substituindo os valores nulos por media
dados_treino['RoomService'] = dados_treino['RoomService'].fillna(dados_treino['RoomService'].median())
dados_treino['FoodCourt'] = dados_treino['FoodCourt'].fillna(dados_treino['FoodCourt'].median())
dados_treino['ShoppingMall'] = dados_treino['ShoppingMall'].fillna(dados_treino['ShoppingMall'].median())
dados_treino['VRDeck'] = dados_treino['VRDeck'].fillna(dados_treino['VRDeck'].median())
dados_treino['Spa'] = dados_treino['Spa'].fillna(dados_treino['Spa'].median())


#substituindo os valores nulos pelos valores mais comuns
dados_treino['HomePlanet'] = dados_treino['HomePlanet'].fillna('Earth')
dados_treino['Destination'] = dados_treino['Destination'].fillna('TRAPPIST-1e')

#substituindo os valores nulos pelos valores mais comuns
dados_treino['CryoSleep'] = dados_treino['CryoSleep'].fillna(False)
dados_treino['VIP'] = dados_treino['VIP'].fillna(False)

#substituindo os valores nulos pela media de idade
dados_treino['Age'] = dados_treino['Age'].fillna(dados_treino['Age'].mean())

#substituindo os valores nulos pelos valores mais comuns (foi escolhido o G, mas pode ser qualquer outro)
dados_treino['Cabin'] = dados_treino['Cabin'].fillna('G/0/S')

In [8]:
dados_treino.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
PassengerId,,,,,,,,,,,,,
0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [9]:
#criando uma variavel que junta as outras, sendo o total gasto pelos passageiros em coisas extras
dados_treino['Total_spending'] = dados_treino['RoomService'] + dados_treino['FoodCourt']
+ dados_treino['ShoppingMall'] + dados_treino['VRDeck'] + dados_treino['Spa']

PassengerId
0001_01       0.0
0002_01     618.0
0003_01    6764.0
0003_02    3893.0
0004_01     718.0
            ...  
9276_01    1717.0
9278_01       0.0
9279_01    1873.0
9280_01    3588.0
9280_02      12.0
Length: 8693, dtype: float64

In [10]:
#separando o deck do lado das cabines 
dados_treino['Cabin_Side'] = dados_treino['Cabin'].str.split("/").str[2]
dados_treino['Cabin_Deck'] = dados_treino['Cabin'].str.split("/").str[0]
dados_treino = dados_treino.drop('Cabin',axis=1)

In [11]:
#tirando os dados nulos restantes e tirando os nomes
dados_treino = dados_treino.dropna()
dados_treino.drop("Name",axis="columns",inplace=True)

In [12]:
#mostrando como o dataframe ficou, tratando os nulos e separando variaveis
dados_treino.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Total_spending,Cabin_Side,Cabin_Deck
PassengerId,,,,,,,,,,,,,,
0001_01,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,0.0,P,B
0002_01,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,118.0,S,F
0003_01,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,3619.0,S,A
0003_02,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,1283.0,S,A
0004_01,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True,373.0,S,F


In [13]:
dados_treino['HomePlanet'] = dados_treino['HomePlanet'].astype('category')
dados_treino['Destination'] = dados_treino['Destination'].astype('category')
dados_treino['Cabin_Deck'] = dados_treino['Cabin_Deck'].astype('category')
dados_treino['Cabin_Side'] = dados_treino['Cabin_Side'].astype('category')

## dimensionalizando os dados categoricos

In [14]:
#transormando as colunas categoricas em colunas categoricas unidimensionais(usando um dummies)
dados_treino = pd.get_dummies(dados_treino,columns=['HomePlanet','Destination','Cabin_Deck','Cabin_Side'])
dados_treino.columns

Index(['CryoSleep', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall',
       'Spa', 'VRDeck', 'Transported', 'Total_spending', 'HomePlanet_Earth',
       'HomePlanet_Europa', 'HomePlanet_Mars', 'Destination_55 Cancri e',
       'Destination_PSO J318.5-22', 'Destination_TRAPPIST-1e', 'Cabin_Deck_A',
       'Cabin_Deck_B', 'Cabin_Deck_C', 'Cabin_Deck_D', 'Cabin_Deck_E',
       'Cabin_Deck_F', 'Cabin_Deck_G', 'Cabin_Deck_T', 'Cabin_Side_P',
       'Cabin_Side_S'],
      dtype='object')

In [15]:
#vendo como ficou o dataframe com os dados categoricos em cada coluna
dados_treino.head()

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Total_spending,...,Cabin_Deck_A,Cabin_Deck_B,Cabin_Deck_C,Cabin_Deck_D,Cabin_Deck_E,Cabin_Deck_F,Cabin_Deck_G,Cabin_Deck_T,Cabin_Side_P,Cabin_Side_S
PassengerId,,,,,,,,,,,,,,,,,,,,,
0001_01,False,39.0,False,0.0,0.0,0.0,0.0,0.0,False,0.0,...,0,1,0,0,0,0,0,0,1,0
0002_01,False,24.0,False,109.0,9.0,25.0,549.0,44.0,True,118.0,...,0,0,0,0,0,1,0,0,0,1
0003_01,False,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,3619.0,...,1,0,0,0,0,0,0,0,0,1
0003_02,False,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,1283.0,...,1,0,0,0,0,0,0,0,0,1
0004_01,False,16.0,False,303.0,70.0,151.0,565.0,2.0,True,373.0,...,0,0,0,0,0,1,0,0,0,1


# escolhendo os parametros do xgboost

In [16]:
#como nao sabemos o melhor parametro para o xgboost, passamos varios, entao importamos uma biblioteca que vai testar e pegar o melhor parametro para o nosso modelo

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
params={
 "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
    
}

In [17]:
#importando o algoritmo classificador xgboost
import xgboost
#inicializando o xgboost
classifier=xgboost.XGBClassifier()

In [18]:
#aqui estamos usando nossa função para pegar os melhores parametros e 
random_search=RandomizedSearchCV(classifier,param_distributions=params,n_iter=5,scoring='roc_auc',n_jobs=-1,cv=5,verbose=3)

In [19]:
#treinando o modelo  com os parametros aleatorios
random_search.fit(dados_treino.drop('Transported',axis='columns'),dados_treino['Transported'])

Fitting 5 folds for each of 5 candidates, totalling 25 fits


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, gamma=None,
                                           gpu_id=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=None, max_bin=None,...
                                           n_estimators=100, n_jobs=None,
                                           num_parallel_tree=Non

In [20]:
#vendo os melhores estimadores para nosso modelo
random_search.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.4,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0.4, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.2, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [21]:
#vendo os melhores parametros que nos colocamos la em cima
random_search.best_params_

{'min_child_weight': 1,
 'max_depth': 5,
 'learning_rate': 0.2,
 'gamma': 0.4,
 'colsample_bytree': 0.4}

# usando o xgboost 

In [22]:
from numpy import nan #importando o nan do numpy
#copiando o output de melhores estimadores e colocando o classificador pra rodar com todos esses parametros
classifier=xgboost.XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.5,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0.0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.15, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=4, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1)

In [23]:
from sklearn.model_selection import KFold

In [24]:
#fazendo a validação cruzada pra score
from sklearn.model_selection import cross_val_score
score=cross_val_score(classifier,dados_treino.drop('Transported',axis='columns'),dados_treino['Transported'],cv=10)
score

array([0.80235294, 0.77882353, 0.79411765, 0.80565371, 0.79387515,
       0.80800942, 0.83156655, 0.8138987 , 0.80212014, 0.80094229])

In [25]:
#media dos 10 scores
score.mean()

0.8031360077599945

In [26]:
#treinando o modelo
xgboost=classifier.fit(dados_treino.drop('Transported',axis='columns'),dados_treino['Transported'])

In [27]:
dados_teste= pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
dados_teste.set_index('PassengerId', inplace=True)
#substituindo os valores nulos por media
dados_teste['RoomService'] = dados_teste['RoomService'].fillna(dados_teste['RoomService'].median())
dados_teste['FoodCourt'] = dados_teste['FoodCourt'].fillna(dados_teste['FoodCourt'].median())
dados_teste['ShoppingMall'] = dados_teste['ShoppingMall'].fillna(dados_teste['ShoppingMall'].median())
dados_teste['VRDeck'] = dados_teste['VRDeck'].fillna(dados_teste['VRDeck'].median())
dados_teste['Spa'] = dados_teste['Spa'].fillna(dados_teste['Spa'].median())


#substituindo os valores nulos pelos valores mais comuns
dados_teste['HomePlanet'] = dados_teste['HomePlanet'].fillna('Earth')
dados_teste['Destination'] = dados_teste['Destination'].fillna('TRAPPIST-1e')

#substituindo os valores nulos pelos valores mais comuns
dados_teste['CryoSleep'] = dados_teste['CryoSleep'].fillna(False)
dados_teste['VIP'] = dados_teste['VIP'].fillna(False)

#substituindo os valores nulos pela media de idade
dados_teste['Age'] = dados_teste['Age'].fillna(dados_teste['Age'].mean())

#substituindo os valores nulos pelos valores mais comuns (foi escolhido o G, mas pode ser qualquer outro)
dados_teste['Cabin'] = dados_teste['Cabin'].fillna('G/0/S')

#criando uma variavel que junta as outras, sendo o total gasto pelos passageiros em coisas extras
dados_teste['Total_spending'] = dados_teste['RoomService'] + dados_teste['FoodCourt']
+ dados_teste['ShoppingMall'] + dados_teste['VRDeck'] + dados_teste['Spa']

#separando o deck do lado das cabines 
dados_teste['Cabin_Side'] = dados_teste['Cabin'].str.split("/").str[2]
dados_teste['Cabin_Deck'] = dados_teste['Cabin'].str.split("/").str[0]
dados_teste = dados_teste.drop('Cabin',axis=1)

#tirando os dados nulos restantes e tirando os nomes

dados_teste.drop("Name",axis="columns",inplace=True)

dados_teste['HomePlanet'] = dados_teste['HomePlanet'].astype('category')
dados_teste['Destination'] = dados_teste['Destination'].astype('category')
dados_teste['Cabin_Deck'] = dados_teste['Cabin_Deck'].astype('category')
dados_teste['Cabin_Side'] = dados_teste['Cabin_Side'].astype('category')

#transormando as colunas categoricas em colunas categoricas unidimensionais(usando um dummies)
dados_teste = pd.get_dummies(dados_teste,columns=['HomePlanet','Destination','Cabin_Deck','Cabin_Side'])
dados_teste.columns

Index(['CryoSleep', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall',
       'Spa', 'VRDeck', 'Total_spending', 'HomePlanet_Earth',
       'HomePlanet_Europa', 'HomePlanet_Mars', 'Destination_55 Cancri e',
       'Destination_PSO J318.5-22', 'Destination_TRAPPIST-1e', 'Cabin_Deck_A',
       'Cabin_Deck_B', 'Cabin_Deck_C', 'Cabin_Deck_D', 'Cabin_Deck_E',
       'Cabin_Deck_F', 'Cabin_Deck_G', 'Cabin_Deck_T', 'Cabin_Side_P',
       'Cabin_Side_S'],
      dtype='object')

# dando o predict

In [28]:
#dando predict nos dados de teste e alocando isso em uma variavel
pred=xgboost.predict(dados_teste)

In [29]:
len(pred)

4277

In [30]:
#colocando esses dados em um dataframe pa entrega
output = pd.DataFrame({'Transported':pred.astype(bool)},index=dados_teste.index)
output.to_csv('submission.csv')
print("Sua entrega foi feita com secesso!")

Sua entrega foi feita com secesso!
